In [29]:
### import packages for "Training" linear regression
import io
import os
import gzip
import pickle
import numpy as np
import pandas as pd
import urllib.request

import matplotlib.pyplot as plt
%matplotlib inline


In [30]:
### import Amazon sagemaker packages

import boto3 
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_deserializer

In [1]:
#
#    Upload training file to S3 from Notebook github repository
#



import boto3
from botocore.exceptions import NoCredentialsError

ACCESS_KEY = '<<ACCESS KEY>>' #arg
SECRET_KEY = '<<SECRET KEY>>' #arg

def upload_to_aws(local_file, bucket, s3_file):
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)

    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

uploaded = upload_to_aws('iris.csv', 'iris-trainb3119a0e-42d2-4ac4-a9e8-f3458356f096', 'iris.csv')  ##args


Upload Successful


In [50]:
#
#   Read in training file
#


import boto3
# import numpy and pandas libraries for working with data
import numpy as np
import pandas as pd

bucket = "iris-trainb3119a0e-42d2-4ac4-a9e8-f3458356f096"
file_name = "iris.csv"

s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)
# 's3' is a key word. create connection to S3 using default config and all buckets within S3

obj = s3.get_object(Bucket= bucket, Key= file_name) 
# get object and file (key) from bucket

train_set = pd.read_csv(obj['Body'])

In [62]:
type(train_set)
data = train_set.loc[:,['sepal_length', 'sepal_width','petal_length','petal_width','species']]
data.head(10)


modelData = np.array(data.iloc[:,0:3]).astype('float32')
target = np.array(data.iloc[:,4]).astype('float32')
#print(modelData[0])

ValueError: could not convert string to float: 'virginica'

In [34]:
bucket = 'iris-trainb3119a0e-42d2-4ac4-a9e8-f3458356f096'
key = "iris.csv"

resource = boto3.resource('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)

bucket_obj=resource.Bucket(bucket)
object_res=bucket_obj.Object(os.path.join(key))



In [35]:
#
#   Location of training data
#

train_data_loc = 's3://{}/{}'.format(bucket, key)
print('Uploaded training data location: {}'.format(train_data_loc))

Uploaded training data location: s3://iris-trainb3119a0e-42d2-4ac4-a9e8-f3458356f096/iris.csv


In [17]:
#
#   Location of training data
#

bucket = 'iris-model-artifactsd506bcfd-74e6-437c-aefb-5c74b6dacd24'

output_location = 's3://{}'.format(bucket)
print('Uploaded training data location: {}'.format(output_location))

Uploaded training data location: s3://iris-model-artifactsd506bcfd-74e6-437c-aefb-5c74b6dacd24


In [36]:
#
#   Use built-in ML in region code is executing
#

region = boto3.Session().region_name
region

'us-east-1'

In [37]:
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner:latest'}
container = containers[region]

In [45]:
sess = sagemaker.Session()


In [46]:
role = get_execution_role()
role

'arn:aws:iam::697525789718:role/service-role/AmazonSageMaker-ExecutionRole-20191019T150576'

In [47]:
linear = sagemaker.estimator.Estimator(container,
                                       role, 
                                       train_instance_count=1, #arg
                                       train_instance_type='ml.c4.xlarge',  #arg
                                       output_path=output_location,
                                       sagemaker_session=sess)

In [48]:
linear.set_hyperparameters(feature_dim=5,
                           predictor_type='regressor')

In [49]:
linear.fit({'train': train_data_loc})

2019-11-04 01:24:23 Starting - Starting the training job...
2019-11-04 01:24:30 Starting - Launching requested ML instances......
2019-11-04 01:25:31 Starting - Preparing the instances for training......
2019-11-04 01:26:52 Downloading - Downloading input data...
2019-11-04 01:27:08 Training - Downloading the training image..Docker entrypoint called with argument(s): train
[11/04/2019 01:27:28 INFO 139966780733248] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u

UnexpectedStatusException: Error for Training job linear-learner-2019-11-04-01-24-22-808: Failed. Reason: ClientError: Unable to read data channel 'train'. Requested content-type is 'application/x-recordio-protobuf'. Please verify the data matches the requested content-type. (caused by MXNetError)

Caused by: [01:27:28] /opt/brazil-pkg-cache/packages/AIAlgorithmsCppLibs/AIAlgorithmsCppLibs-2.0.2051.0/AL2012/generic-flavor/src/src/aialgs/io/iterator_base.cpp:100: (Input Error) The header of the MXNet RecordIO record at position 0 in the dataset does not start with a valid magic number.

Stack trace returned 10 entries:
[bt] (0) /opt/amazon/lib/libaialgs.so(+0xb1f0) [0x7f4c8c8401f0]
[bt] (1) /opt/amazon/lib/libaialgs.so(+0xb54a) [0x7f4c8c84054a]
[bt] (2) /opt/amazon/lib/libaialgs.so(aialgs::iterator_base::Next()+0x4a6) [0x7f4c8c849436]
[bt] (3) /opt/amazon/lib/libmxnet.so(MXDataIterNext+0x21) [0x7f4c7b31f131]
[bt] (4) /opt/amazon/python2.7/lib/python2.7/lib-dynload/_ctypes.so(ffi_call_un